## 專題：成功大學測量所-房價預估
## Landuse : Commercial_Res

In [1]:
import pandas as pd
import numpy as np
import shapefile
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import distance
import os
import re

# 請先修改檔案路徑

In [2]:
# 讀取檔案路徑設定
path = '/Users/tc.chen/Documents/data_projncku/projectdata/nckugeo01/'
Landuse = 'Landuse_area/'
savefile = '/Users/tc.chen/Documents/data_projncku/projectdata/save/'

### 查回資料  -- 請下載 hub 上的檔案
Basic : tgos_80k.csv 

In [3]:
# 匯入新查回資料 並 篩選 Response_X有資料者
Basic = pd.read_csv( path + 'tgos_80k.csv', index_col = 0)
Basic = Basic[Basic['Response_X'].isnull() == False]

In [4]:
Basic.head(2)

,index,鄉鎮市區,交易標的,土地區段位置/建物區段門牌,土地移轉總面積(平方公尺),都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,備註,編號,登記日期,Address_x,id,Address_y,Response_Address,Response_X,Response_Y,csv_file_name
level_0,,,,,,,,,,,,,,,,,,,,,
0,0,南區,房地(土地+建物),臺中市南區復興路二段31~60號,2.82,商,NaN,NaN,1061029,土地1建物1車位0,...,NaN,RPRPMLOKJIMFFAB57CA,2018S1,臺中市南區復興路二段46號,1.0,臺中市南區復興路二段46號,臺中市南區和平里3鄰復興路二段46號,215528.254,2668405.642,1.0
1,1,中區,房地(土地+建物),臺中市中區成功路306巷1~30號,9.62,商,NaN,NaN,1061018,土地1建物2車位0,...,NaN,RPPNMLPKJIMFFAB27CA,2018S1,臺中市中區成功路306巷16號,2.0,臺中市中區成功路306巷16號,臺中市中區大誠里19鄰成功路306巷10號,217371.292,2671123.323,1.0


In [5]:
len(Basic)

78685

In [6]:
# 確認 Response_X, Response_y 的欄位位置
Basic.iloc[1, 34:36]

Response_X         217371
Response_Y    2.67112e+06
Name: 1, dtype: object

In [7]:
Basic.columns

Index(['index', '鄉鎮市區', '交易標的', '土地區段位置/建物區段門牌', '土地移轉總面積(平方公尺)', '都市土地使用分區',
       '非都市土地使用分區', '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態',
       '主要用途', '主要建材', '建築完成年月', '建物移轉總面積(平方公尺)', '建物現況格局-房', '建物現況格局-廳',
       '建物現況格局-衛', '建物現況格局-隔間', '有無管理組織', '總價(元)', '單價(元/平方公尺)', '車位類別',
       '車位移轉總面積(平方公尺)', '車位總價(元)', '備註', '編號', '登記日期', 'Address_x', 'id',
       'Address_y', 'Response_Address', 'Response_X', 'Response_Y',
       'csv_file_name'],
      dtype='object')

### 物件周邊土地利用面積
(2) Landuse : 計算物件周邊土地利用(住宅/工業/商業)等相關面積

In [8]:
# 匯入Landuse相關檔案

# 修改當前工作目錄
os.chdir(path + Landuse)

# 確認工作目錄下之檔案清單列表
file_list = os.listdir()
print(file_list)

['.DS_Store', 'All_REs.cpg', 'All_REs.dbf', 'All_REs.sbn', 'All_REs.sbx', 'All_REs.shp', 'All_REs.shp.xml', 'All_REs.shx', 'Commercial_Res.CPG', 'Commercial_Res.dbf', 'Commercial_Res.sbn', 'Commercial_Res.sbx', 'Commercial_Res.shp', 'Commercial_Res.shp.xml', 'Commercial_Res.shx', 'desktop.ini', 'Indus_Res.cpg', 'Indus_Res.dbf', 'Indus_Res.sbn', 'Indus_Res.sbx', 'Indus_Res.shp', 'Indus_Res.shp.xml', 'Indus_Res.shx', 'Mix_res.cpg', 'Mix_res.dbf', 'Mix_res.sbn', 'Mix_res.sbx', 'Mix_res.shp', 'Mix_res.shp.xml', 'Mix_res.shx', 'Pure_residential.cpg', 'Pure_residential.dbf', 'Pure_residential.sbn', 'Pure_residential.sbx', 'Pure_residential.shp', 'Pure_residential.shp.xml', 'Pure_residential.shx', 'read me.txt']


In [9]:
# 匯入商用住宅 .shp檔案
# All_Res = gpd.GeoDataFrame.from_file(path + Landuse + 'All_REs.shp')
Commercial_Res = gpd.GeoDataFrame.from_file(path + Landuse + 'Commercial_Res.shp')
# Indus_Res = gpd.GeoDataFrame.from_file(path + Landuse + 'Indus_Res.shp')
# Mix_Res = gpd.GeoDataFrame.from_file(path + Landuse + 'Mix_res.shp')
# Pure_Res = gpd.GeoDataFrame.from_file(path + Landuse + 'Pure_residential.shp')

# 請先確認Radius後輸入各自負責的即可執行下面的程式

## Radius 分配
> Alvin radius = [100, 200]

> Yoga radius = [500]

> Nick radius = [1000]

> Rita radius  = [1500]

> Allen radius = [2000]

In [ ]:
radius = [1500] ## 請輸入各自負責的(數值)
batch_size = 10000 ## 目前設定一批為10000筆資料

ii = 0
while(ii + batch_size) <= len(Basic):
    Basic_temp = Basic.iloc[ii:ii + batch_size, :]        
    for k in radius:
        Commercial_Res_list = []
        Commercial_area = pd.DataFrame()
        column_name = 'Commercial_Res_' + str(k)
        file_name = 'Commercial_Res_' + str(k) + '_data' + str(ii + batch_size)
        file_name1 = 'Commercial_Res_' + str(k) + '_data' + '_combine'
            
        for j in tqdm(range(len(Basic_temp))):
            temp_Real_Data=pd.DataFrame(Basic_temp.iloc[j, 34:36])
            temp_Real_Data=temp_Real_Data.transpose()
            temp_Real_Data_Point = Point(temp_Real_Data.Response_X, temp_Real_Data.Response_Y).buffer(k)
            d = 0 #最後要計算總面積用的
            for i in range(len(Commercial_Res.geometry)):
                b = Commercial_Res.geometry[i]
                c = temp_Real_Data_Point.intersection(b)
                d += c.area
            Commercial_Res_list.append(d)        
        Commercial_Res_pd = pd.DataFrame(Commercial_Res_list)
        Commercial_Res_pd.columns = [column_name]
        Commercial_Res_pd['orignal_index'] = Basic_temp.index ## 保留原本的index 避免與原始檔案串接後錯亂(有發現index不連續)
        Commercial_area = pd.concat([Commercial_area, Commercial_Res_pd], axis = 1)
        Commercial_area.to_csv(savefile + file_name)
        Commercial_area.to_csv(savefile + file_name1, 
                               mode ='a',
                               header = False)       
    ii += batch_size

HBox(children=(IntProgress(value=0, max=10000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=10000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=10000), HTML(value='')))